## Machine Learning project


In [5]:
pip install -r requirements.txt

  Cloning https://github.com/chlorochrule/cknn to /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-req-build-91ulfn55
  Running command git clone --filter=blob:none --quiet https://github.com/chlorochrule/cknn /private/var/folders/dc/1bymglpd6198p4cqkyfcnrqw0000gn/T/pip-req-build-91ulfn55
  Resolved https://github.com/chlorochrule/cknn to commit 7d05c5049da72a573bd486fca6647f8b0376243c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


Note: you may need to restart the kernel to use updated packages.


### Embedded-Space


### MI-net

In [1]:
import numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [4]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'fp':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch][0] = result[1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set..astype(np.float32)
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch][0] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net(dataset):
    """Train and evaluate on MI-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # features pooling
    fp = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp')(dropout)

    model = Model(inputs=[data_input], outputs=[fp])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc

In [5]:
# perform five times 10-fold cross-validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=np.float32)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = MI_Net(dataset[ifold])
print('MI-Net mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

run= 0   fold= 0
epoch= 0   train_loss= 2.963   train_acc= 0.573   test_loss=3.051   test_acc= 0.300
epoch= 1   train_loss= 2.726   train_acc= 0.768   test_loss=2.876   test_acc= 0.700
epoch= 2   train_loss= 2.562   train_acc= 0.902   test_loss=2.802   test_acc= 0.700
epoch= 3   train_loss= 2.486   train_acc= 0.915   test_loss=2.670   test_acc= 0.900
epoch= 4   train_loss= 2.385   train_acc= 0.951   test_loss=2.698   test_acc= 0.700
epoch= 5   train_loss= 2.364   train_acc= 0.976   test_loss=2.616   test_acc= 0.900
epoch= 6   train_loss= 2.309   train_acc= 0.988   test_loss=2.605   test_acc= 0.900
epoch= 7   train_loss= 2.305   train_acc= 0.963   test_loss=2.635   test_acc= 0.700
epoch= 8   train_loss= 2.239   train_acc= 0.976   test_loss=2.522   test_acc= 0.900
epoch= 9   train_loss= 2.219   train_acc= 0.988   test_loss=2.574   test_acc= 0.700
epoch= 10   train_loss= 2.224   train_acc= 0.963   test_loss=2.443   test_acc= 0.900
epoch= 11   train_loss= 2.178   train_acc= 0.988   test_lo

epoch= 46   train_loss= 1.706   train_acc= 1.000   test_loss=1.914   test_acc= 0.900
epoch= 47   train_loss= 1.696   train_acc= 1.000   test_loss=1.907   test_acc= 0.900
epoch= 48   train_loss= 1.686   train_acc= 1.000   test_loss=1.902   test_acc= 0.900
epoch= 49   train_loss= 1.678   train_acc= 1.000   test_loss=1.889   test_acc= 0.900
run time: 0.7238585313161214 min
test_acc=0.900
run= 0   fold= 2
epoch= 0   train_loss= 2.857   train_acc= 0.675   test_loss=2.853   test_acc= 0.889
epoch= 1   train_loss= 2.585   train_acc= 0.892   test_loss=2.785   test_acc= 0.556
epoch= 2   train_loss= 2.485   train_acc= 0.916   test_loss=2.744   test_acc= 0.889
epoch= 3   train_loss= 2.393   train_acc= 0.976   test_loss=2.715   test_acc= 0.889
epoch= 4   train_loss= 2.349   train_acc= 0.976   test_loss=2.674   test_acc= 0.889
epoch= 5   train_loss= 2.280   train_acc= 1.000   test_loss=2.688   test_acc= 0.889
epoch= 6   train_loss= 2.294   train_acc= 0.952   test_loss=2.695   test_acc= 0.889
epoch= 

epoch= 42   train_loss= 1.755   train_acc= 1.000   test_loss=1.829   test_acc= 1.000
epoch= 43   train_loss= 1.740   train_acc= 1.000   test_loss=1.819   test_acc= 1.000
epoch= 44   train_loss= 1.732   train_acc= 1.000   test_loss=1.812   test_acc= 1.000
epoch= 45   train_loss= 1.721   train_acc= 1.000   test_loss=1.796   test_acc= 1.000
epoch= 46   train_loss= 1.712   train_acc= 1.000   test_loss=1.785   test_acc= 1.000
epoch= 47   train_loss= 1.707   train_acc= 1.000   test_loss=1.783   test_acc= 1.000
epoch= 48   train_loss= 1.697   train_acc= 1.000   test_loss=1.772   test_acc= 1.000
epoch= 49   train_loss= 1.683   train_acc= 1.000   test_loss=1.770   test_acc= 1.000
run time: 0.9416592796643575 min
test_acc=1.000
run= 0   fold= 4
epoch= 0   train_loss= 2.927   train_acc= 0.687   test_loss=2.666   test_acc= 1.000
epoch= 1   train_loss= 2.641   train_acc= 0.855   test_loss=2.574   test_acc= 1.000
epoch= 2   train_loss= 2.540   train_acc= 0.904   test_loss=2.477   test_acc= 1.000
epo

epoch= 38   train_loss= 1.780   train_acc= 1.000   test_loss=2.024   test_acc= 0.889
epoch= 39   train_loss= 1.771   train_acc= 1.000   test_loss=1.988   test_acc= 0.889
epoch= 40   train_loss= 1.761   train_acc= 1.000   test_loss=1.988   test_acc= 0.889
epoch= 41   train_loss= 1.756   train_acc= 1.000   test_loss=2.004   test_acc= 0.889
epoch= 42   train_loss= 1.738   train_acc= 1.000   test_loss=1.972   test_acc= 0.889
epoch= 43   train_loss= 1.728   train_acc= 1.000   test_loss=1.962   test_acc= 0.889
epoch= 44   train_loss= 1.717   train_acc= 1.000   test_loss=1.959   test_acc= 0.889
epoch= 45   train_loss= 1.708   train_acc= 1.000   test_loss=1.939   test_acc= 0.889
epoch= 46   train_loss= 1.698   train_acc= 1.000   test_loss=1.928   test_acc= 0.889
epoch= 47   train_loss= 1.690   train_acc= 1.000   test_loss=1.932   test_acc= 0.889
epoch= 48   train_loss= 1.679   train_acc= 1.000   test_loss=1.916   test_acc= 0.889
epoch= 49   train_loss= 1.667   train_acc= 1.000   test_loss=1.90

epoch= 34   train_loss= 1.830   train_acc= 1.000   test_loss=1.891   test_acc= 1.000
epoch= 35   train_loss= 1.816   train_acc= 1.000   test_loss=1.888   test_acc= 1.000
epoch= 36   train_loss= 1.803   train_acc= 1.000   test_loss=1.875   test_acc= 1.000
epoch= 37   train_loss= 1.794   train_acc= 1.000   test_loss=1.866   test_acc= 1.000
epoch= 38   train_loss= 1.782   train_acc= 1.000   test_loss=1.852   test_acc= 1.000
epoch= 39   train_loss= 1.774   train_acc= 1.000   test_loss=1.834   test_acc= 1.000
epoch= 40   train_loss= 1.765   train_acc= 1.000   test_loss=1.833   test_acc= 1.000
epoch= 41   train_loss= 1.755   train_acc= 1.000   test_loss=1.813   test_acc= 1.000
epoch= 42   train_loss= 1.740   train_acc= 1.000   test_loss=1.807   test_acc= 1.000
epoch= 43   train_loss= 1.730   train_acc= 1.000   test_loss=1.796   test_acc= 1.000
epoch= 44   train_loss= 1.722   train_acc= 1.000   test_loss=1.783   test_acc= 1.000
epoch= 45   train_loss= 1.710   train_acc= 1.000   test_loss=1.77

...

epoch= 10   train_loss= 2.175   train_acc= 1.000   test_loss=2.330   test_acc= 0.889
epoch= 11   train_loss= 2.176   train_acc= 1.000   test_loss=2.320   test_acc= 1.000
epoch= 12   train_loss= 2.138   train_acc= 1.000   test_loss=2.277   test_acc= 1.000
epoch= 13   train_loss= 2.132   train_acc= 1.000   test_loss=2.276   test_acc= 1.000
epoch= 14   train_loss= 2.105   train_acc= 1.000   test_loss=2.245   test_acc= 1.000
epoch= 15   train_loss= 2.090   train_acc= 1.000   test_loss=2.215   test_acc= 1.000
epoch= 16   train_loss= 2.073   train_acc= 1.000   test_loss=2.205   test_acc= 1.000
epoch= 17   train_loss= 2.053   train_acc= 1.000   test_loss=2.189   test_acc= 1.000
epoch= 18   train_loss= 2.039   train_acc= 1.000   test_loss=2.173   test_acc= 1.000
epoch= 19   train_loss= 2.027   train_acc= 1.000   test_loss=2.148   test_acc= 1.000
epoch= 20   train_loss= 2.011   train_acc= 1.000   test_loss=2.139   test_acc= 1.000
epoch= 21   train_loss= 2.000   train_acc= 1.000   test_loss=2.12

epoch= 6   train_loss= 2.279   train_acc= 0.988   test_loss=2.470   test_acc= 0.900
epoch= 7   train_loss= 2.252   train_acc= 0.988   test_loss=2.393   test_acc= 1.000
epoch= 8   train_loss= 2.211   train_acc= 1.000   test_loss=2.381   test_acc= 1.000
epoch= 9   train_loss= 2.201   train_acc= 1.000   test_loss=2.341   test_acc= 1.000
epoch= 10   train_loss= 2.178   train_acc= 1.000   test_loss=2.342   test_acc= 1.000
epoch= 11   train_loss= 2.153   train_acc= 1.000   test_loss=2.342   test_acc= 0.900
epoch= 12   train_loss= 2.139   train_acc= 1.000   test_loss=2.288   test_acc= 1.000
epoch= 13   train_loss= 2.126   train_acc= 1.000   test_loss=2.271   test_acc= 1.000
epoch= 14   train_loss= 2.111   train_acc= 1.000   test_loss=2.292   test_acc= 0.900
epoch= 15   train_loss= 2.089   train_acc= 1.000   test_loss=2.265   test_acc= 1.000
epoch= 16   train_loss= 2.072   train_acc= 1.000   test_loss=2.224   test_acc= 1.000
epoch= 17   train_loss= 2.056   train_acc= 1.000   test_loss=2.217   

epoch= 2   train_loss= 2.524   train_acc= 0.940   test_loss=2.547   test_acc= 1.000
epoch= 3   train_loss= 2.438   train_acc= 0.964   test_loss=2.457   test_acc= 1.000
epoch= 4   train_loss= 2.369   train_acc= 0.976   test_loss=2.478   test_acc= 0.889
epoch= 5   train_loss= 2.368   train_acc= 0.952   test_loss=2.425   test_acc= 0.889
epoch= 6   train_loss= 2.289   train_acc= 0.988   test_loss=2.322   test_acc= 1.000
epoch= 7   train_loss= 2.268   train_acc= 1.000   test_loss=2.318   test_acc= 1.000
epoch= 8   train_loss= 2.244   train_acc= 0.988   test_loss=2.312   test_acc= 0.889
epoch= 9   train_loss= 2.221   train_acc= 0.988   test_loss=2.294   test_acc= 0.889
epoch= 10   train_loss= 2.194   train_acc= 1.000   test_loss=2.240   test_acc= 1.000
epoch= 11   train_loss= 2.177   train_acc= 1.000   test_loss=2.213   test_acc= 1.000
epoch= 12   train_loss= 2.154   train_acc= 1.000   test_loss=2.227   test_acc= 0.889
epoch= 13   train_loss= 2.132   train_acc= 1.000   test_loss=2.215   test

epoch= 48   train_loss= 1.687   train_acc= 1.000   test_loss=2.149   test_acc= 0.889
epoch= 49   train_loss= 1.677   train_acc= 1.000   test_loss=2.143   test_acc= 0.889
run time: 1.0385619163513184 min
test_acc=0.889
run= 2   fold= 5
epoch= 0   train_loss= 2.884   train_acc= 0.675   test_loss=2.794   test_acc= 0.778
epoch= 1   train_loss= 2.610   train_acc= 0.928   test_loss=2.760   test_acc= 1.000
epoch= 2   train_loss= 2.519   train_acc= 0.916   test_loss=2.714   test_acc= 1.000
epoch= 3   train_loss= 2.437   train_acc= 0.940   test_loss=2.592   test_acc= 1.000
epoch= 4   train_loss= 2.405   train_acc= 0.940   test_loss=2.550   test_acc= 1.000
epoch= 5   train_loss= 2.332   train_acc= 0.976   test_loss=2.521   test_acc= 1.000
epoch= 6   train_loss= 2.306   train_acc= 0.976   test_loss=2.491   test_acc= 0.889
epoch= 7   train_loss= 2.265   train_acc= 0.988   test_loss=2.563   test_acc= 0.778
epoch= 8   train_loss= 2.257   train_acc= 0.952   test_loss=2.479   test_acc= 0.889
epoch= 9 

KeyboardInterrupt: 

### MI-net pooling deep supervision

In [7]:
import numpy as np
import sys
import time
import random
from random import shuffle
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout, average

from mil_nets.dataset import load_dataset
from mil_nets.layer import Feature_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [9]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net with deep supervision model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'fp1':batch[1].astype(np.float32), 'fp2':batch[1].astype(np.float32), 'fp3':batch[1].astype(np.float32), 'ave':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch] = result[-1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training MI-Net with deep supervision model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'fp1':batch[1].astype(np.float32), 'fp2':batch[1].astype(np.float32), 'fp3':batch[1].astype(np.float32), 'ave':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[-1]
    return np.mean(train_loss), np.mean(train_acc)

def MI_Net_with_DS(dataset):
    """Train and evaluate on MI-Net with deep supervision.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of MI-Net with deep supervision.
    """
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]
    weight = [1.0, 1.0, 1.0, 0.0]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout1 = Dropout(rate=0.5)(fc1)
    dropout2 = Dropout(rate=0.5)(fc2)
    dropout3 = Dropout(rate=0.5)(fc3)

    # features pooling
    fp1 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp1')(dropout1)
    fp2 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp2')(dropout2)
    fp3 = Feature_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='fp3')(dropout3)

    # score average
    mg_ave =average([fp1,fp2,fp3], name='ave')

    model = Model(inputs=[data_input], outputs=[fp1, fp2, fp3, mg_ave])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss={'fp1':bag_loss, 'fp2':bag_loss, 'fp3':bag_loss, 'ave':bag_loss}, loss_weights={'fp1':weight[0], 'fp2':weight[1], 'fp3':weight[2], 'ave':weight[3]}, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc

In [ ]:
# perform five times 10-fold cross=validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=float)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = MI_Net_with_DS(dataset[ifold])
print('MI-Net with DS mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

run= 0   fold= 0
epoch= 0   train_loss= 4.697   train_acc= 0.646   test_loss=3.608   test_acc= 0.700
epoch= 1   train_loss= 3.576   train_acc= 0.817   test_loss=3.062   test_acc= 0.900
epoch= 2   train_loss= 2.949   train_acc= 0.927   test_loss=3.044   test_acc= 0.900
epoch= 3   train_loss= 2.781   train_acc= 0.939   test_loss=2.858   test_acc= 0.900
epoch= 4   train_loss= 2.590   train_acc= 0.976   test_loss=2.990   test_acc= 0.900
epoch= 5   train_loss= 2.591   train_acc= 0.951   test_loss=2.754   test_acc= 1.000
epoch= 6   train_loss= 2.410   train_acc= 0.988   test_loss=2.781   test_acc= 0.900
epoch= 7   train_loss= 2.364   train_acc= 1.000   test_loss=2.655   test_acc= 1.000
epoch= 8   train_loss= 2.309   train_acc= 1.000   test_loss=2.594   test_acc= 1.000
epoch= 9   train_loss= 2.276   train_acc= 1.000   test_loss=2.758   test_acc= 0.900
epoch= 10   train_loss= 2.267   train_acc= 1.000   test_loss=2.562   test_acc= 1.000
epoch= 11   train_loss= 2.248   train_acc= 1.000   test_lo

## Bag-Space

*CkNN*

https://github.com/chlorochrule/cknn/blob/master/cknn/cknn.py

In [ ]:
import pandas as pd
X = pd.read_table("./clean2.data") #pd.read_csv("sample_data/mnist_test.csv") 

In [ ]:
from cknn import cknneighbors_graph

#ckng = cknneighbors_graph(X, n_neighbors=5, delta=1.0)

In [ ]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns

from cknn import cknneighbors_graph

sns.set()


def plot2d_label(X, title=None):
    digits = load_digits()
    y = digits.target
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()


def main():
    data = X
    print(data)
    n_neighbors = 2

    model_normal = SpectralEmbedding(n_components=2, n_neighbors=n_neighbors)
    y_normal = model_normal.fit_transform(data)
    plot2d_label(y_normal)

    #ckng = cknneighbors_graph(data, n_neighbors=n_neighbors, delta=1.5)
    #model_cknn = SpectralEmbedding(n_components=2, affinity='precomputed')
    #y_cknn = model_cknn.fit_transform(ckng.toarray())
    #plot2d_label(y_cknn)

main()

     MUSK-211,211_1+1,46,-108,-60,-69,-117,49,38,-161,-8,5,-323,-220,-113,-299,-283,-307,-31,-106,-227,-42,-59,-22,-67,189,81,17,-27,-89,-67,105,-116,124,-106,5,-120,63,-165,40,-27,68,-44,98,-33,-314,-282,-335,-144,-13,-197,-2,-144,-13,-11,-131,108,-43,42,-151,-4,8,-102,51,-15,108,-135,59,-166,20,-20,23,-48,-68,-299,-256,-97,-183,-24,-271,-229,-177,-6,0,-129,112,15,36,-66,-54,-75,132,-188,119,-120,-312,23,-55,-53,-26,-71,41,-55,148,-247,-306,-308,-230,-166,-35,-205,-280,-239,-53,-10,-23,25,-5,163,61,59,-39,92,72,113,-107,80,25,-27,81,-114,-187,45,-118,-75,-182,-234,-19,12,-13,-41,-119,-149,70,17,-20,-177,-101,-116,-14,-50,24,-81,-125,-114,-44,128,3,-244,-308,52,-7,39,126,156,-50,-112,96,1.
0     MUSK-211,211_1+10,41,-188,-145,22,-117,-6,57,-...                                                                                                                                                                                                                                                      

ValueError: ignored

In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import SpectralEmbedding
import matplotlib.pyplot as plt
from matplotlib import offsetbox
import seaborn as sns

sns.set()


def plot2d_label(X, title=None):
    y = X[1]
    x_min, x_max = np.min(X[0], 0), np.max(X[0], 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    plt.xticks([])
    plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.show()


def main():
    data = X
    n_neighbors = 10

    model_normal = SpectralEmbedding(n_components=2, n_neighbors=n_neighbors)
    y_normal = model_normal.fit_transform(data)
    #plot2d_label(y_normal)

    ckng = cknneighbors_graph(data, n_neighbors=n_neighbors, delta=1.5)
    model_cknn = SpectralEmbedding(n_components=2, affinity='precomputed')
    y_cknn = model_cknn.fit_transform(ckng.toarray())
    #plot2d_label(y_cknn)
    print(y_cknn)

main()

NameError: name 'X' is not defined

## instance-Space


### mi-Net

In [1]:
import sys
import time
from random import shuffle
import numpy as np
import argparse

from keras.models import Model
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.layers import Input, Dense, Layer, Dropout

from mil_nets.dataset import load_dataset
from mil_nets.layer import Score_pooling
from mil_nets.metrics import bag_accuracy
from mil_nets.objectives import bag_loss
from mil_nets.utils import convertToBatch

In [22]:
def test_eval(model, test_set):
    """Evaluate on testing set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    test_set : list
        A list of testing set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on testing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_test_batch = len(test_set)
    test_loss = np.zeros((num_test_batch, 1), dtype=np.float32)
    test_acc = np.zeros((num_test_batch, 1), dtype=np.float32)
    for ibatch, batch in enumerate(test_set):
        result = model.test_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        test_loss[ibatch] = result[0]
        test_acc[ibatch] = result[1]
    return np.mean(test_loss), np.mean(test_acc)

def train_eval(model, train_set):
    """Evaluate on training set.
    Parameters
    -----------------
    model : keras.engine.training.Model object
        The training mi-Net model.
    train_set : list
        A list of training set contains all training bags features and labels.
    Returns
    -----------------
    test_loss : float
        Mean loss of evaluating on traing set.
    test_acc : float
        Mean accuracy of evaluating on testing set.
    """
    num_train_batch = len(train_set)
    train_loss = np.zeros((num_train_batch, 1), dtype=np.float32)
    train_acc = np.zeros((num_train_batch, 1), dtype=np.float32)
    shuffle(train_set)
    for ibatch, batch in enumerate(train_set):
        result = model.train_on_batch({'input':batch[0].astype(np.float32)}, {'sp':batch[1].astype(np.float32)})
        train_loss[ibatch] = result[0]
        train_acc[ibatch] = result[1]
    return np.mean(train_loss), np.mean(train_acc)

def mi_Net(dataset):
    weight_decay=0.005
    init_lr=5e-4
    pooling_mode='max'
    momentum=0.9
    max_epoch=50
    """Train and evaluate on mi-Net.
    Parameters
    -----------------
    dataset : dict
        A dictionary contains all dataset information. We split train/test by keys.
    Returns
    -----------------
    test_acc : float
        Testing accuracy of mi-Net.
    """
    # load data and convert type
    train_bags = dataset['train']
    test_bags = dataset['test']

    # convert bag to batch
    train_set = convertToBatch(train_bags)
    test_set = convertToBatch(test_bags)
    dimension = train_set[0][0].shape[1]

    # data: instance feature, n*d, n = number of training instance
    data_input = Input(shape=(dimension,), dtype='float32', name='input')

    # fully-connected
    fc1 = Dense(256, activation='relu', kernel_regularizer=l2(weight_decay))(data_input)
    fc2 = Dense(128, activation='relu', kernel_regularizer=l2(weight_decay))(fc1)
    fc3 = Dense(64, activation='relu', kernel_regularizer=l2(weight_decay))(fc2)

    # dropout
    dropout = Dropout(rate=0.5)(fc3)

    # score pooling
    sp = Score_pooling(output_dim=1, kernel_regularizer=l2(weight_decay), pooling_mode=pooling_mode, name='sp')(dropout)

    model = Model(inputs=[data_input], outputs=[sp])
    sgd = SGD(lr=init_lr, decay=1e-4, momentum=momentum, nesterov=True)
    model.compile(loss=bag_loss, optimizer=sgd, metrics=[bag_accuracy])

    # train model
    t1 = time.time()
    num_batch = len(train_set)
    for epoch in range(max_epoch):
        train_loss, train_acc = train_eval(model, train_set)
        test_loss, test_acc = test_eval(model, test_set)
        print('epoch=', epoch, '  train_loss= {:.3f}'.format(train_loss), '  train_acc= {:.3f}'.format(train_acc), '  test_loss={:.3f}'.format(test_loss), '  test_acc= {:.3f}'.format(test_acc))
    t2 = time.time()
    print('run time:', (t2-t1) / 60.0, 'min')
    print('test_acc={:.3f}'.format(test_acc))

    return test_acc


In [23]:
# perform five times 10-fold cross-validation experiments
run = 5
n_folds = 10
acc = np.zeros((run, n_folds), dtype=np.float32)
for irun in range(run):
    dataset = load_dataset('musk1', n_folds)
    for ifold in range(n_folds):
        print('run=', irun, '  fold=', ifold)
        acc[irun][ifold] = mi_Net(dataset[ifold])
print('mi-net mean accuracy = ', np.mean(acc))
print('std = ', np.std(acc))

MUSK-188     
MUSK-188     
MUSK-188     
MUSK-188     
MUSK-190     
MUSK-190     
MUSK-190     
MUSK-190     
MUSK-211     
MUSK-211     
MUSK-212     
MUSK-212     
MUSK-212     
MUSK-213     
MUSK-213     
MUSK-213     
MUSK-213     
MUSK-219     
MUSK-219     
MUSK-224     
MUSK-224     
MUSK-227     
MUSK-227     
MUSK-228     
MUSK-228     
MUSK-228     
MUSK-228     
MUSK-228     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-236     
MUSK-238     
MUSK-238     
MUSK-238     
MUSK-238     
MUSK-238     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-240     
MUSK-246     
MUSK-246     
MUSK-246     
MUSK-246     
MUSK-254     
MUSK-254     
MUSK-256     
MUSK-256     
MUSK-256     
MUSK-256     
MUSK-272     
MUSK-272     
MUSK-272     
MUSK-273     
MUSK-273     
MUSK-273     
MUSK-273     
MUSK-273     
MUSK-284     
MUSK-284     
MUSK-284     
MUSK-284     
MUSK-285     
MUSK-285     
MUSK-2

epoch= 18   train_loss= 2.081   train_acc= 1.000   test_loss=2.426   test_acc= 0.900
epoch= 19   train_loss= 2.059   train_acc= 1.000   test_loss=2.396   test_acc= 0.900
epoch= 20   train_loss= 2.065   train_acc= 0.976   test_loss=2.386   test_acc= 0.900
epoch= 21   train_loss= 2.039   train_acc= 0.976   test_loss=2.394   test_acc= 0.900
epoch= 22   train_loss= 2.029   train_acc= 1.000   test_loss=2.429   test_acc= 0.900
epoch= 23   train_loss= 2.010   train_acc= 0.976   test_loss=2.376   test_acc= 0.900
epoch= 24   train_loss= 1.978   train_acc= 1.000   test_loss=2.356   test_acc= 0.900
epoch= 25   train_loss= 1.962   train_acc= 1.000   test_loss=2.370   test_acc= 0.900
epoch= 26   train_loss= 1.946   train_acc= 1.000   test_loss=2.366   test_acc= 0.900
epoch= 27   train_loss= 1.941   train_acc= 1.000   test_loss=2.373   test_acc= 0.900
epoch= 28   train_loss= 1.922   train_acc= 1.000   test_loss=2.357   test_acc= 0.900
epoch= 29   train_loss= 1.907   train_acc= 1.000   test_loss=2.33

epoch= 14   train_loss= 2.198   train_acc= 0.976   test_loss=2.442   test_acc= 0.778
epoch= 15   train_loss= 2.180   train_acc= 0.976   test_loss=2.355   test_acc= 0.778
epoch= 16   train_loss= 2.147   train_acc= 0.964   test_loss=2.329   test_acc= 0.778
epoch= 17   train_loss= 2.141   train_acc= 0.988   test_loss=2.366   test_acc= 0.667
epoch= 18   train_loss= 2.141   train_acc= 0.964   test_loss=2.397   test_acc= 0.778
epoch= 19   train_loss= 2.089   train_acc= 0.988   test_loss=2.366   test_acc= 0.778
epoch= 20   train_loss= 2.079   train_acc= 0.976   test_loss=2.333   test_acc= 0.778
epoch= 21   train_loss= 2.063   train_acc= 0.976   test_loss=2.295   test_acc= 0.778
epoch= 22   train_loss= 2.044   train_acc= 1.000   test_loss=2.254   test_acc= 0.778
epoch= 23   train_loss= 2.030   train_acc= 0.976   test_loss=2.209   test_acc= 0.889
epoch= 24   train_loss= 2.014   train_acc= 1.000   test_loss=2.221   test_acc= 0.778
epoch= 25   train_loss= 1.978   train_acc= 1.000   test_loss=2.21

epoch= 10   train_loss= 2.322   train_acc= 0.940   test_loss=2.306   test_acc= 0.889
epoch= 11   train_loss= 2.300   train_acc= 0.952   test_loss=2.276   test_acc= 0.889
epoch= 12   train_loss= 2.265   train_acc= 0.964   test_loss=2.326   test_acc= 0.889
epoch= 13   train_loss= 2.245   train_acc= 0.976   test_loss=2.291   test_acc= 0.889
epoch= 14   train_loss= 2.179   train_acc= 0.988   test_loss=2.261   test_acc= 0.889
epoch= 15   train_loss= 2.181   train_acc= 0.952   test_loss=2.224   test_acc= 0.889
epoch= 16   train_loss= 2.165   train_acc= 0.976   test_loss=2.276   test_acc= 0.889
epoch= 17   train_loss= 2.138   train_acc= 0.976   test_loss=2.307   test_acc= 0.889
epoch= 18   train_loss= 2.099   train_acc= 0.988   test_loss=2.221   test_acc= 0.889
epoch= 19   train_loss= 2.072   train_acc= 1.000   test_loss=2.234   test_acc= 0.889
epoch= 20   train_loss= 2.062   train_acc= 1.000   test_loss=2.244   test_acc= 0.889
epoch= 21   train_loss= 2.043   train_acc= 0.988   test_loss=2.19

epoch= 6   train_loss= 2.495   train_acc= 0.867   test_loss=2.423   test_acc= 0.889
epoch= 7   train_loss= 2.424   train_acc= 0.928   test_loss=2.344   test_acc= 1.000
epoch= 8   train_loss= 2.427   train_acc= 0.880   test_loss=2.349   test_acc= 0.889
epoch= 9   train_loss= 2.384   train_acc= 0.940   test_loss=2.274   test_acc= 1.000
epoch= 10   train_loss= 2.336   train_acc= 0.916   test_loss=2.262   test_acc= 1.000
epoch= 11   train_loss= 2.274   train_acc= 0.964   test_loss=2.232   test_acc= 1.000
epoch= 12   train_loss= 2.259   train_acc= 0.940   test_loss=2.262   test_acc= 0.889
epoch= 13   train_loss= 2.225   train_acc= 0.964   test_loss=2.160   test_acc= 1.000
epoch= 14   train_loss= 2.190   train_acc= 0.964   test_loss=2.307   test_acc= 0.889
epoch= 15   train_loss= 2.192   train_acc= 0.952   test_loss=2.262   test_acc= 0.889
epoch= 16   train_loss= 2.154   train_acc= 0.988   test_loss=2.142   test_acc= 1.000
epoch= 17   train_loss= 2.122   train_acc= 0.988   test_loss=2.128   

...

epoch= 6   train_loss= 2.474   train_acc= 0.928   test_loss=2.486   test_acc= 0.778
epoch= 7   train_loss= 2.408   train_acc= 0.916   test_loss=2.483   test_acc= 0.778
epoch= 8   train_loss= 2.386   train_acc= 0.916   test_loss=2.438   test_acc= 0.778
epoch= 9   train_loss= 2.366   train_acc= 0.916   test_loss=2.483   test_acc= 0.778
epoch= 10   train_loss= 2.306   train_acc= 0.952   test_loss=2.484   test_acc= 0.778
epoch= 11   train_loss= 2.306   train_acc= 0.940   test_loss=2.488   test_acc= 0.778
epoch= 12   train_loss= 2.229   train_acc= 0.976   test_loss=2.457   test_acc= 0.778
epoch= 13   train_loss= 2.238   train_acc= 0.940   test_loss=2.411   test_acc= 0.778
epoch= 14   train_loss= 2.187   train_acc= 0.988   test_loss=2.457   test_acc= 0.778
epoch= 15   train_loss= 2.175   train_acc= 0.964   test_loss=2.387   test_acc= 0.778
epoch= 16   train_loss= 2.158   train_acc= 0.988   test_loss=2.475   test_acc= 0.778
epoch= 17   train_loss= 2.160   train_acc= 0.952   test_loss=2.373   

epoch= 2   train_loss= 2.699   train_acc= 0.855   test_loss=2.813   test_acc= 0.667
epoch= 3   train_loss= 2.655   train_acc= 0.807   test_loss=2.719   test_acc= 0.778
epoch= 4   train_loss= 2.545   train_acc= 0.880   test_loss=2.697   test_acc= 0.889
epoch= 5   train_loss= 2.500   train_acc= 0.904   test_loss=2.745   test_acc= 0.778
epoch= 6   train_loss= 2.469   train_acc= 0.892   test_loss=2.643   test_acc= 0.778
epoch= 7   train_loss= 2.425   train_acc= 0.904   test_loss=2.623   test_acc= 0.778
epoch= 8   train_loss= 2.401   train_acc= 0.904   test_loss=2.608   test_acc= 0.778
epoch= 9   train_loss= 2.341   train_acc= 0.952   test_loss=2.649   test_acc= 0.778
epoch= 10   train_loss= 2.309   train_acc= 0.940   test_loss=2.651   test_acc= 0.667
epoch= 11   train_loss= 2.284   train_acc= 0.940   test_loss=2.650   test_acc= 0.778
epoch= 12   train_loss= 2.252   train_acc= 0.952   test_loss=2.635   test_acc= 0.778
epoch= 13   train_loss= 2.202   train_acc= 0.976   test_loss=2.656   test

epoch= 48   train_loss= 1.698   train_acc= 1.000   test_loss=1.844   test_acc= 0.889
epoch= 49   train_loss= 1.693   train_acc= 1.000   test_loss=1.871   test_acc= 0.889
run time: 0.7094895680745442 min
test_acc=0.889
run= 4   fold= 6
epoch= 0   train_loss= 2.918   train_acc= 0.711   test_loss=2.843   test_acc= 0.778
epoch= 1   train_loss= 2.835   train_acc= 0.711   test_loss=2.783   test_acc= 0.667
epoch= 2   train_loss= 2.721   train_acc= 0.819   test_loss=2.692   test_acc= 0.889
epoch= 3   train_loss= 2.669   train_acc= 0.807   test_loss=2.647   test_acc= 0.778
epoch= 4   train_loss= 2.582   train_acc= 0.867   test_loss=2.607   test_acc= 0.889
epoch= 5   train_loss= 2.572   train_acc= 0.855   test_loss=2.546   test_acc= 0.889
epoch= 6   train_loss= 2.479   train_acc= 0.892   test_loss=2.504   test_acc= 0.889
epoch= 7   train_loss= 2.506   train_acc= 0.855   test_loss=2.493   test_acc= 0.889
epoch= 8   train_loss= 2.419   train_acc= 0.940   test_loss=2.471   test_acc= 0.889
epoch= 9 

epoch= 44   train_loss= 1.732   train_acc= 1.000   test_loss=3.326   test_acc= 0.778
epoch= 45   train_loss= 1.726   train_acc= 1.000   test_loss=3.319   test_acc= 0.778
epoch= 46   train_loss= 1.710   train_acc= 1.000   test_loss=3.318   test_acc= 0.778
epoch= 47   train_loss= 1.705   train_acc= 1.000   test_loss=3.291   test_acc= 0.778
epoch= 48   train_loss= 1.696   train_acc= 1.000   test_loss=3.289   test_acc= 0.778
epoch= 49   train_loss= 1.684   train_acc= 1.000   test_loss=3.270   test_acc= 0.778
run time: 0.7162322839101155 min
test_acc=0.778
run= 4   fold= 8
epoch= 0   train_loss= 3.014   train_acc= 0.627   test_loss=2.805   test_acc= 0.889
epoch= 1   train_loss= 2.784   train_acc= 0.735   test_loss=2.753   test_acc= 0.778
epoch= 2   train_loss= 2.655   train_acc= 0.855   test_loss=2.701   test_acc= 0.778
epoch= 3   train_loss= 2.597   train_acc= 0.867   test_loss=2.692   test_acc= 0.778
epoch= 4   train_loss= 2.503   train_acc= 0.880   test_loss=2.694   test_acc= 0.778
epoch

epoch= 40   train_loss= 1.807   train_acc= 1.000   test_loss=1.882   test_acc= 1.000
epoch= 41   train_loss= 1.796   train_acc= 1.000   test_loss=1.873   test_acc= 1.000
epoch= 42   train_loss= 1.787   train_acc= 1.000   test_loss=1.834   test_acc= 1.000
epoch= 43   train_loss= 1.777   train_acc= 1.000   test_loss=1.833   test_acc= 1.000
epoch= 44   train_loss= 1.771   train_acc= 1.000   test_loss=1.803   test_acc= 1.000
epoch= 45   train_loss= 1.756   train_acc= 1.000   test_loss=1.808   test_acc= 1.000
epoch= 46   train_loss= 1.751   train_acc= 1.000   test_loss=1.796   test_acc= 1.000
epoch= 47   train_loss= 1.734   train_acc= 1.000   test_loss=1.791   test_acc= 1.000
epoch= 48   train_loss= 1.733   train_acc= 1.000   test_loss=1.797   test_acc= 1.000
epoch= 49   train_loss= 1.719   train_acc= 1.000   test_loss=1.780   test_acc= 1.000
run time: 0.7182445844014486 min
test_acc=1.000
mi-net mean accuracy =  0.8300000059604645
std =  0.10402396848155698
